# connie-skipper

Repository at [https://github.com/PhMota/connie-skipper](https://github.com/PhMota/connie-skipper)

In [1]:
%load_ext autoreload
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
# import gc
from glob import glob
import xarray as xr
import connie_skipper as sk
import warnings
warnings.simplefilter("ignore", category=xr.core.extensions.AccessorRegistrationWarning)

In [2]:
from IPython.display import display, IFrame, clear_output
from ipywidgets import interact, interactive_output
import ipywidgets as widgets

In [3]:
class App:
    def __init__(self, rootpath):
        self.output = widgets.Output()
        files = glob(rootpath+"*/*.fz")
        excl = glob(rootpath+"*/*sho_noreset*.fz")
        self.select = widgets.Select( 
            options = [ (file.split(rootpath)[-1], file) for file in sorted( set(files) - set(excl) )[::-1] ], 
            layout = {'width': '90%'},
            rows = 10
        )
        self.progress = widgets.IntProgress(
            value = 0, 
            min = 0, 
            max = 10, 
            description = 'loading'
        )
        self.progressPlot = widgets.IntProgress(
            value = 0, 
            min = 0, 
            max = 10, 
            description = 'plotting'
        )
        self.app = None
        with self.output:
            display(self.select, self.progress, self.progressPlot)
        display(self.output)
        self.select.observe( 
            lambda path: self.plot_from_path2(path.new),
            names = "value"
        )
        self.interactive_figure = None
        self.plot_from_path2(self.select.value)
        
    def plot_from_path2(self, path):
        self.progress.value, self.progress.description = 1, "opening file"
        self.da = sk.fits_to_DataArray( path )

        self.progress.value, self.progress.description = 5, "widgets"
        chid_max = len(self.da["chid"].data)-1
        samp_max = int(self.da["samp"].data[-1])
        app_dict = {}
        if self.app is None:
            trim = widgets.Checkbox(
                description = 'trim', 
                value = True
            )
            demod = widgets.Checkbox(
                description = 'demod/osi', 
                value = True
            )
            statsProj = widgets.Dropdown( 
                options = ["median", "mean", "std", "mad", "min", "max"], 
                description = 'stats(proj)' 
            )
            median_row = widgets.Checkbox(
                description = 'median(row)',
                value = True
            )
            median_col = widgets.Checkbox(description='median(col)')
            spectrum = widgets.Checkbox(description='spectrum', value=True)
            chid = widgets.IntSlider(
                value = 0,
                min = 0,
                max = chid_max,
                description = f'chid[0─{chid_max}]',
                continuous_update = False
            )
            samp = widgets.IntSlider( 
                value = 0,
                min = 0, 
                max = samp_max,
                description = f'samp[0─{samp_max}]',
                continuous_update = False
            )
            stats = widgets.Dropdown( options=["none", "median", "mean", "std", "mad", "min", "max"], description='stats(samp)' )
            stats.value = "mean"
            statsRange = widgets.IntRangeSlider(
                value = [0, samp_max+1],
                min = 0,
                max = samp_max+1,
                description = f"{stats.value}",
                continuous_update = False
            )
            energyFactor = widgets.IntSlider(
                value = 100,
                min = 0,
                max = 200,
                description = "µE ± σE",
                continuous_update = False
            )
            
            stats.observe( lambda opt: setattr(statsRange, "description", f"{opt.new}(samp)"), names="value" )
            stats.observe( 
                lambda opt: setattr(statsRange, "disable", True) if opt.new == "none" else setattr(statsRange, "disable", False),
                names="value" 
            )
            ui = widgets.VBox([ trim, demod, statsProj, median_row, median_col, spectrum, chid, samp, stats, statsRange, energyFactor])
            app_dict = {
                "median_row": median_row, 
                "median_col": median_col, 
                "spectrum": spectrum,
                "trim": trim, 
                "demod": demod, 
                "statsProj": statsProj, 
                "chid": chid, 
                "samp": samp, 
                "stats": stats, 
                "stats_range": statsRange,
                "energy_percentile": energyFactor
            } 
        else:
            self.app["chid"].max = chid_max
            self.app["chid"].description = f'chid[0─{chid_max}]'
            self.app["samp"].max = samp_max
            self.app["samp"].description = f'samp[0─{samp_max}]'
            self.app["stats_range"].value = [0, samp_max+1]
            self.app["stats_range"].max = samp_max+1
            app_dict = self.app

        self.progress.value, self.progress.description = 7, "plotting"
        plot = interactive_output( 
            self.gen_plot, 
            app_dict
        )
        self.progress.value, self.progress.description = 9, "finishing"
        if self.app is None:
            with self.output:
                display( widgets.HBox([ui,plot]) )
            self.app = app_dict
        self.progress.value, self.progress.description = 10, "complete"
        
    def gen_plot(self, 
        median_row, 
        median_col, 
        spectrum,
        demod,
        trim, 
        chid, 
        samp, 
        stats, 
        stats_range,
        statsProj,
        energy_percentile,
    ):
#         if self.interactive_figure is not None:
#             self.interactive_figure.clf()
        
        op = lambda _da: _da.isel(chid=chid)
        if stats == "none":
            stats_str = stats
            op = lambda _da, prevop=op: prevop(_da).isel(samp=samp)
        else:
            stats_str = f"{stats}({stats_range[0]}, {stats_range[1]})"
            op = lambda _da, prevop=op: prevop(_da).isel(samp=slice(*stats_range)).skipper.stats(dim = "samp", mode = stats)
        if trim:
            op = lambda _da, prevop=op: prevop(_da).skipper.trim()
        if demod:
            op = lambda _da, prevop=op: prevop(_da).skipper.demodulate("col")
        self.interactive_figure = op(self.da).skipper.plot_full( 
            robust = True, 
            fig = self.interactive_figure, 
            title = "test",
            xproj = median_row,
            yproj = median_col,
            spectrum = spectrum,
            energy_percentile = energy_percentile,
            mode = statsProj,
            suptitle = f"chid={chid} samp={samp} stats={stats_str}",
            progress_bar = (self.progressPlot, 0, 10)
        )

App("/share/storage2/connie/data/lta/testAngra/");

Output()

In [9]:
a = widgets.Select
dir(a)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_notifiers',
 '_call_widget_constructed',
 '_comm_changed',
 '_compare',
 '_default_keys',
 '_display_callbacks',
 '_dom_classes',
 '_gen_repr_from_keys',
 '_get_embed_state',
 '_handle_custom_msg',
 '_handle_displayed',
 '_handle_msg',
 '_holding_sync',
 '_ipython_display_',
 '_is_numpy',
 '_lock_property',
 '_log_default',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_msg_callbacks',
 '_notify_trait',
 '_options_full',
 '_options_labels',
 '_propagate_index',
 '_propagate_label',
 '_propagate_options',
 '_propagate_value',
 '_property_lock',
 '_register